In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np

from ml_model import run_model
from ml_model import long_only_strategy_daily
from ml_model import long_only_strategy_monthly

/Users/hongyangyang/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
df = pd.read_excel('analysis.xlsx',"Sheet1")

In [4]:
df.columns

Index(['date', 'ticker', 'monthly_return', 'paper_month', 'total_count',
       'total_count_cumsum', 'total_citation', 'total_citation_cumsum',
       'patent_count', 'patent_count_cumsum', 'patent_citation',
       'patent_citation_cumsum', 'journal_count', 'journal_count_cumsum',
       'journal_citation', 'journal_citation_cumsum', 'conference_count',
       'conference_count_cumsum', 'conference_citation',
       'conference_citation_cumsum', 'Articles_count', 'Articles_count_cumsum',
       'Articles_citation', 'Articles_citation_cumsum', 'Books_count',
       'Books_count_cumsum', 'Books_citation', 'Books_citation_cumsum',
       'total_average_citation', 'total_max_citation',
       'patent_average_citation', 'patent_max_citation',
       'journal_average_citation', 'journal_max_citation',
       'conference_average_citation', 'conference_max_citation',
       'Articles_average_citation', 'Articles_max_citation',
       'Books_average_citation', 'Books_max_citation'],
      dty

In [5]:
features_column = ['total_count',
       'total_count_cumsum', 'total_citation', 'total_citation_cumsum',
       'patent_count', 'patent_count_cumsum', 'patent_citation',
       'patent_citation_cumsum', 'journal_count', 'journal_count_cumsum',
       'journal_citation', 'journal_citation_cumsum', 'conference_count',
       'conference_count_cumsum', 'conference_citation',
       'conference_citation_cumsum', 'Articles_count', 'Articles_count_cumsum',
       'Articles_citation', 'Articles_citation_cumsum', 'Books_count',
       'Books_count_cumsum', 'Books_citation', 'Books_citation_cumsum',
       'total_average_citation', 'total_max_citation',
       'patent_average_citation', 'patent_max_citation',
       'journal_average_citation', 'journal_max_citation',
       'conference_average_citation', 'conference_max_citation',
       'Articles_average_citation', 'Articles_max_citation',
       'Books_average_citation', 'Books_max_citation']

# Prepare Data

## 1. Get Unique Ticker

In [6]:
unique_ticker = sorted(df.ticker.unique())

In [7]:
len(unique_ticker)

93

## 2. Get Unique Datatime (monthly)

In [8]:
unique_datetime=[x.strftime("%Y-%m-%d") for x in df.date.dt.date.unique()]

In [9]:
len(unique_datetime)

108

In [10]:
len(unique_datetime)/12

9.0

In [11]:
unique_datetime[42]

'2013-07-31'

## 3. Get Trade Month

In [12]:
trade_month = unique_datetime[42:]

In [13]:
len(trade_month)

66

## 4. Get tic Monthly Return

In [14]:
tic_monthly_return=df.pivot('date','ticker','monthly_return')
tic_monthly_return=tic_monthly_return.reset_index()
tic_monthly_return.index=tic_monthly_return.date
del tic_monthly_return['date']

In [15]:
tic_monthly_return=tic_monthly_return[tic_monthly_return.index>=unique_datetime[42]]

In [16]:
tic_monthly_return.shape

(66, 93)

In [17]:
#tic_monthly_return.to_excel("tic_monthly_return.xlsx","Sheet1")

## 5. Get Equally Portfolio Monthly Return

In [18]:
equally_portfolio_return=[]
for i in range(len(trade_month)):
    return_remove_nan = tic_monthly_return.iloc[i][~np.isnan(tic_monthly_return.iloc[i])]
    equally_portfolio_return.append(sum(return_remove_nan)/len(return_remove_nan))


In [19]:
df_equally_portfolio_return=pd.DataFrame(equally_portfolio_return,trade_month)
df_equally_portfolio_return=df_equally_portfolio_return.reset_index()
df_equally_portfolio_return.columns= ['trade_month','monthly_return']
df_equally_portfolio_return.index=df_equally_portfolio_return.trade_month
df_equally_portfolio_return=df_equally_portfolio_return['monthly_return']



In [20]:
df_equally_portfolio_return.shape

(66,)

## 6. Get Daily price

In [21]:
daily_price = pd.read_csv("adj_close_price.csv")
daily_price.index = daily_price.date
daily_price=daily_price[unique_ticker]

In [22]:
#daily_return = daily_price.pct_change()

daily_return = (daily_price.shift(-1)-daily_price)/daily_price
daily_return = daily_return[daily_return.index>=trade_month[0]]

In [23]:
daily_return.shape

(1397, 93)

## 7. Get Trade Month for daily return

In [24]:
trade_month_plus1=trade_month.copy()
trade_month_plus1.append('2019-01-31')

In [25]:
len(trade_month)

66

In [26]:
len(trade_month_plus1)

67

## Run Model
* Input: Features + Forward Month Return
* Output: Monthly Stock Predicted Return 


In [ ]:
%%time

model_results= run_model(df,unique_ticker,unique_datetime,trade_month,features_column,first_trade_date_index = 42,testing_windows = 6)


Trade Month:  2013-07-31
Trade Month:  2013-08-30
Trade Month:  2013-09-30
Trade Month:  2013-10-31
Trade Month:  2013-11-29
Trade Month:  2013-12-31
Trade Month:  2014-01-31
Trade Month:  2014-02-28
Trade Month:  2014-03-31
Trade Month:  2014-04-30
Trade Month:  2014-05-30
Trade Month:  2014-06-30
Trade Month:  2014-07-31
Trade Month:  2014-08-29
Trade Month:  2014-09-30
Trade Month:  2014-10-31
Trade Month:  2014-11-28
Trade Month:  2014-12-31
Trade Month:  2015-01-30


## Output: 1. Get Predicted Return

In [ ]:
df_predict_lr = model_results[0].astype(np.float64)
df_predict_rf = model_results[1].astype(np.float64)
df_predict_lasso = model_results[2].astype(np.float64)
df_predict_gbm = model_results[3].astype(np.float64)
df_predict_ada = model_results[4].astype(np.float64)
df_predict_best = model_results[5].astype(np.float64)
df_best_model_name = model_results[6]
df_evaluation_score = model_results[7]

In [ ]:
#df_predict_lasso.to_excel('df_predict_lasso.xlsx','Sheet1')
#df_predict_rf.to_excel('df_predict_rf.xlsx','Sheet1')
#df_predict_best.to_excel('df_predict_best.xlsx','Sheet1')
#df_predict_gbm.to_excel('df_predict_gbm.xlsx','Sheet1')



In [ ]:
df_best_model_name.model_name.value_counts()

## Output: 2. Get Portfolio Daily Return

In [ ]:
df_portfolio_lr_daily=long_only_strategy_daily(df_predict_lr,daily_return,trade_month_plus1,top_quantile_threshold = 0.8)
df_portfolio_rf_daily=long_only_strategy_daily(df_predict_rf,daily_return,trade_month_plus1,top_quantile_threshold = 0.8)
df_portfolio_lasso_daily=long_only_strategy_daily(df_predict_lasso,daily_return,trade_month_plus1,top_quantile_threshold = 0.8)
df_portfolio_gbm_daily=long_only_strategy_daily(df_predict_gbm,daily_return,trade_month_plus1,top_quantile_threshold = 0.8)
df_portfolio_ada_daily=long_only_strategy_daily(df_predict_ada,daily_return,trade_month_plus1,top_quantile_threshold = 0.8)

df_portfolio_best_daily=long_only_strategy_daily(df_predict_best,daily_return,trade_month_plus1,top_quantile_threshold = 0.8)


In [ ]:
df_portfolio_lr_daily.to_excel('df_portfolio_lr_daily.xlsx',"Sheet1")
df_portfolio_rf_daily.to_excel('df_portfolio_rf_daily.xlsx',"Sheet1")
df_portfolio_lasso_daily.to_excel('df_portfolio_lasso_daily.xlsx',"Sheet1")
df_portfolio_gbm_daily.to_excel('df_portfolio_gbm_daily.xlsx',"Sheet1")
df_portfolio_ada_daily.to_excel('df_portfolio_ada_daily.xlsx',"Sheet1")
df_portfolio_best_daily.to_excel('df_portfolio_best_daily.xlsx',"Sheet1")


## Output: 3. Get Portfolio Monthly Return

In [ ]:
df_portfolio_return_lr=long_only_strategy_monthly(df_predict_lr,tic_monthly_return,trade_month,top_quantile_threshold = 0.8)
df_portfolio_return_rf=long_only_strategy_monthly(df_predict_rf,tic_monthly_return,trade_month,top_quantile_threshold = 0.8)
df_portfolio_return_lasso=long_only_strategy_monthly(df_predict_lasso,tic_monthly_return,trade_month,top_quantile_threshold = 0.8)
df_portfolio_return_gbm=long_only_strategy_monthly(df_predict_gbm,tic_monthly_return,trade_month,top_quantile_threshold = 0.8)
df_portfolio_return_ada=long_only_strategy_monthly(df_predict_ada,tic_monthly_return,trade_month,top_quantile_threshold = 0.8)

df_portfolio_return_best=long_only_strategy_monthly(df_predict_best,tic_monthly_return,trade_month,top_quantile_threshold = 0.8)
